In [8]:
from google.colab import drive
from prettytable import PrettyTable
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, r2_score
import numpy as np
import pandas as pd
drive.mount('/content/drive')
df_qlt_train_1 = pd.read_csv('/content/drive/MyDrive/aaa/datasets/db_bench/readrandom_4/qlt.csv')
df_qlt_train_1 = df_qlt_train_1.head(int(np.round(len(df_qlt_train_1) * 0.01)))
df_qlt_train_1.columns = ['time_qlt', 'key', 'type_id', 'cf_id_qlt', 'value_size']
df_bclt_train_1 = pd.read_csv('/content/drive/MyDrive/aaa/datasets/db_bench/readrandom_4/bclt.csv')
df_bclt_train_1 = df_bclt_train_1.head(int(np.round(len(df_bclt_train_1) * 0.01)))
df_qlt_train_2 = pd.read_csv('/content/drive/MyDrive/aaa/datasets/db_bench/read_2/qlt.csv')
df_qlt_train_2 = df_qlt_train_2.head(int(np.round(len(df_qlt_train_2) * 0.01)))
df_qlt_train_2.columns = ['time_qlt', 'key', 'type_id', 'cf_id_qlt', 'value_size']
df_bclt_train_2 = pd.read_csv('/content/drive/MyDrive/aaa/datasets/db_bench/read_2/bclt.csv')
df_bclt_train_2 = df_bclt_train_2.head(int(np.round(len(df_bclt_train_2) * 0.01)))
df_qlt_test = pd.read_csv('/content/drive/MyDrive/aaa/datasets/db_bench/read_1/qlt.csv')
df_qlt_test = df_qlt_test.head(int(np.round(len(df_qlt_test) * 0.01)))
df_qlt_test.columns = ['time_qlt', 'key', 'type_id', 'cf_id_qlt', 'value_size']
df_bclt_test = pd.read_csv('/content/drive/MyDrive/aaa/datasets/db_bench/read_1/bclt.csv')
df_bclt_test = df_bclt_test.head(int(np.round(len(df_bclt_test) * 0.01)))
df_qlt_train_1 = pd.concat([df_qlt_train_1, pd.DataFrame(-1, index=range(abs(len(df_bclt_train_1) - len(df_qlt_train_1))), columns=df_qlt_train_1.columns)], ignore_index=True)
df_bclt_train_1 = pd.concat([df_bclt_train_1, pd.DataFrame(-1, index=range(abs(len(df_bclt_train_1) - len(df_qlt_train_1))), columns=df_bclt_train_1.columns)], ignore_index=True)
df_qlt_train_2 = pd.concat([df_qlt_train_2, pd.DataFrame(-1, index=range(abs(len(df_bclt_train_2) - len(df_qlt_train_2))), columns=df_qlt_train_2.columns)], ignore_index=True)
df_bclt_train_2 = pd.concat([df_bclt_train_2, pd.DataFrame(-1, index=range(abs(len(df_bclt_train_2) - len(df_qlt_train_2))), columns=df_bclt_train_2.columns)], ignore_index=True)
df_qlt_test = pd.concat([df_qlt_test, pd.DataFrame(-1, index=range(abs(len(df_bclt_test) - len(df_qlt_test))), columns=df_qlt_test.columns)], ignore_index=True)
df_bclt_test = pd.concat([df_bclt_test, pd.DataFrame(-1, index=range(abs(len(df_bclt_test) - len(df_qlt_test))), columns=df_bclt_test.columns)], ignore_index=True)
df_1 = pd.concat([df_qlt_train_1, df_bclt_train_1], axis=1)
df_2 = pd.concat([df_qlt_train_2, df_bclt_train_2], axis=1)
df_3 = pd.concat([df_qlt_test, df_bclt_test], axis=1)
data = pd.concat([df_1, df_2, df_3], ignore_index=True)
data = data.reset_index()
test_size = len(df_qlt_test)
train_data, test_data = data[:-test_size], data[-test_size:]
prediction_indices = range(len(train_data), len(train_data) + test_size)
prediction_data = pd.DataFrame(prediction_indices, columns=['index'], index=prediction_indices)
for col in [column for column in train_data.columns if train_data[column].nunique() == 1]:
    prediction_data[col] = pd.Series(train_data[col].iloc[0]).repeat(test_size).values
prediction_data['time_qlt'] = test_data['time_qlt']
prediction_data['key'] = test_data['key']
prediction_data['type_id'] = test_data['type_id']
prediction_data['cf_id_qlt'] = test_data['cf_id_qlt']
prediction_data['value_size'] = test_data['value_size']
columns_not_in_prediction = [col for col in train_data.columns.tolist() if col not in prediction_data.columns.tolist()]
unique_values_dict = {}
for col in columns_not_in_prediction:
    unique_values = train_data[col].nunique()
    unique_values_dict[col] = unique_values
continuous_columns = []
discrete_columns = []
for col, unique_values in unique_values_dict.items():
    if unique_values > 500:
        continuous_columns.append(col)
    else:
        discrete_columns.append(col)
num_rows = int(np.round(len(train_data) * 0.05))
for col in columns_not_in_prediction:
    features = df_qlt_train_1.columns
    X_test = prediction_data[features]
    if col in continuous_columns:
        X_train = train_data[features].values[-num_rows:]
        y_train = train_data[col].values[-num_rows:]
        model = LinearRegression()
    else:
        X_train = train_data[features]
        y_train = train_data[col]
        model = LogisticRegression(max_iter=10000000000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = np.where(y_pred < -1, -1, y_pred)
    if train_data[col].dtype == np.int64:
        y_pred = np.round(y_pred).astype(np.int64)
    prediction_data[col] = y_pred
columns_to_drop = ['index']
data = data.drop(columns_to_drop, axis=1)
train_data = train_data.drop(columns_to_drop, axis=1)
test_data = test_data.drop(columns_to_drop, axis=1)
prediction_data = prediction_data.drop(columns_to_drop, axis=1)
prediction_data = prediction_data[train_data.columns]
table = PrettyTable()
table.field_names = ["Column", "Score", "Metric"]
scores = {}
for col in columns_not_in_prediction:
    if col in continuous_columns:
        score = r2_score(test_data[col], prediction_data[col])
        if score < 0:
            score = 0
        metric = 'R-squared'
    else:
        score = accuracy_score(test_data[col], prediction_data[col])
        metric = 'Accuracy'
    scores[col] = (score, metric)
sorted_scores = sorted(scores.items(), key=lambda x: x[1][0], reverse=True)
for col, (score, metric) in sorted_scores:
  table.add_row([col, f'{(score * 100):.2f}%', metric])
average_score = np.mean([score for score, metric in scores.values()])
table.add_row(["TOTAL", f'{(average_score * 100):.2f}%', 'Total'])
print(table)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


+--------------+--------+-----------+
|    Column    | Score  |   Metric  |
+--------------+--------+-----------+
|  cache_hit   | 50.37% |  Accuracy |
|  block_type  | 50.01% |  Accuracy |
|  data_size   | 50.01% |  Accuracy |
|  key_exist   | 50.01% |  Accuracy |
|    cf_id     | 50.00% |  Accuracy |
|  key_count   | 25.01% |  Accuracy |
|  block_size  | 4.54%  |  Accuracy |
|     sst      | 2.52%  |  Accuracy |
|     time     | 0.00%  | R-squared |
|   block_id   | 0.00%  | R-squared |
|      id      | 0.00%  | R-squared |
|    key_id    | 0.00%  | R-squared |
| block_offset | 0.00%  |  Accuracy |
|    TOTAL     | 21.73% |   Total   |
+--------------+--------+-----------+


In [9]:
data

,time_qlt,key,type_id,cf_id_qlt,value_size,time,block_id,block_type,block_size,cf_id,...,key_id,data_size,cache_hit,key_exist,key_count,table_id,sequence_number,block_key_size,key_size,block_offset
0,1691505003386701,606931,0,1,0,1691505003386978,1,7,123776,4,...,1,0,1,0,0,1,0,16,28,107
1,1691505003386722,762869,0,4,0,1691505003386980,2,7,123776,0,...,2,0,1,0,0,1,0,16,28,107
2,1691505003386744,354598,0,3,0,1691505003386980,3,7,123776,3,...,3,0,1,0,0,1,0,16,28,107
3,1691505003386762,861016,0,1,0,1691505003387042,2,7,123776,0,...,4,0,1,0,0,1,0,16,28,107
4,1691505003386776,688646,0,1,0,1691505003387049,4,7,123776,2,...,5,0,1,0,0,1,0,16,28,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122055,-1,-1,-1,-1,-1,1684914926976471,65,7,281360,0,...,18367,0,0,0,0,1,0,16,28,94
122056,-1,-1,-1,-1,-1,1684914926976490,17891,9,4096,0,...,18363,128,0,1,32,1,0,16,28,94
122057,-1,-1,-1,-1,-1,1684914926976474,17892,9,4096,0,...,18362,128,0,1,32,1,0,16,28,94
122058,-1,-1,-1,-1,-1,1684914926976499,26,7,281360,0,...,18368,0,0,0,0,1,0,16,28,94


In [10]:
train_data

,time_qlt,key,type_id,cf_id_qlt,value_size,time,block_id,block_type,block_size,cf_id,...,key_id,data_size,cache_hit,key_exist,key_count,table_id,sequence_number,block_key_size,key_size,block_offset
0,1691505003386701,606931,0,1,0,1691505003386978,1,7,123776,4,...,1,0,1,0,0,1,0,16,28,107
1,1691505003386722,762869,0,4,0,1691505003386980,2,7,123776,0,...,2,0,1,0,0,1,0,16,28,107
2,1691505003386744,354598,0,3,0,1691505003386980,3,7,123776,3,...,3,0,1,0,0,1,0,16,28,107
3,1691505003386762,861016,0,1,0,1691505003387042,2,7,123776,0,...,4,0,1,0,0,1,0,16,28,107
4,1691505003386776,688646,0,1,0,1691505003387049,4,7,123776,2,...,5,0,1,0,0,1,0,16,28,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85295,-1,-1,-1,-1,-1,1685381028569837,50,7,281328,0,...,24858,0,0,0,0,1,0,16,28,10365
85296,-1,-1,-1,-1,-1,1685381028569841,23908,9,4112,0,...,24855,128,0,1,25,1,0,16,28,10365
85297,-1,-1,-1,-1,-1,1685381028569850,8,7,281376,0,...,24859,0,0,0,0,1,0,16,28,10365
85298,-1,-1,-1,-1,-1,1685381028569863,23909,9,4112,0,...,24857,128,0,1,24,1,0,16,28,10365


In [11]:
test_data

,time_qlt,key,type_id,cf_id_qlt,value_size,time,block_id,block_type,block_size,cf_id,...,key_id,data_size,cache_hit,key_exist,key_count,table_id,sequence_number,block_key_size,key_size,block_offset
85300,1684914926421540,7721212,0,0,0,1684914926422107,1,7,279568,0,...,1,0,1,0,0,1,0,16,28,94
85301,1684914926421618,1879256,0,0,0,1684914926422108,2,7,281344,0,...,2,0,1,0,0,1,0,16,28,94
85302,1684914926421635,3855267,0,0,0,1684914926422674,3,7,281312,0,...,3,0,0,0,0,1,0,16,28,94
85303,1684914926421663,5106527,0,0,0,1684914926422716,4,7,281360,0,...,4,0,0,0,0,1,0,16,28,94
85304,1684914926421713,7964217,0,0,0,1684914926422730,3,7,281312,0,...,5,0,0,0,0,1,0,16,28,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122055,-1,-1,-1,-1,-1,1684914926976471,65,7,281360,0,...,18367,0,0,0,0,1,0,16,28,94
122056,-1,-1,-1,-1,-1,1684914926976490,17891,9,4096,0,...,18363,128,0,1,32,1,0,16,28,94
122057,-1,-1,-1,-1,-1,1684914926976474,17892,9,4096,0,...,18362,128,0,1,32,1,0,16,28,94
122058,-1,-1,-1,-1,-1,1684914926976499,26,7,281360,0,...,18368,0,0,0,0,1,0,16,28,94


In [12]:
prediction_data

,time_qlt,key,type_id,cf_id_qlt,value_size,time,block_id,block_type,block_size,cf_id,...,key_id,data_size,cache_hit,key_exist,key_count,table_id,sequence_number,block_key_size,key_size,block_offset
85300,1684914926421540,7721212,0,0,0,1685381028546114,11108,7,4112,0,...,23769,0,0,0,0,1,0,16,28,10365
85301,1684914926421618,1879256,0,0,0,1685381028546114,11108,7,4112,0,...,23769,0,0,0,0,1,0,16,28,10365
85302,1684914926421635,3855267,0,0,0,1685381028546114,11108,7,4112,0,...,23769,0,0,0,0,1,0,16,28,10365
85303,1684914926421663,5106527,0,0,0,1685381028546114,11108,7,4112,0,...,23769,0,0,0,0,1,0,16,28,10365
85304,1684914926421713,7964217,0,0,0,1685381028546114,11108,7,4112,0,...,23769,0,0,0,0,1,0,16,28,10365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122055,-1,-1,-1,-1,-1,1685381028546114,11108,9,281392,2,...,23769,128,1,1,25,1,0,16,28,107
122056,-1,-1,-1,-1,-1,1685381028546114,11108,9,281392,2,...,23769,128,1,1,25,1,0,16,28,107
122057,-1,-1,-1,-1,-1,1685381028546114,11108,9,281392,2,...,23769,128,1,1,25,1,0,16,28,107
122058,-1,-1,-1,-1,-1,1685381028546114,11108,9,281392,2,...,23769,128,1,1,25,1,0,16,28,107
